# Hyperspy Tutorial

## EDS-TEM quantification of core shell nanoparticles

Using a machine learning techniques, such as independent component analysis (ICA), the composition of buried nanostructures, such as core-shell nanoparticles, can be accurately measured as demonstrated by D. Roussow et al., Nano Letters, 2015 (see the [full article](https://www.repository.cam.ac.uk/bitstream/handle/1810/248102/Roussouw%20et%20al%202015%20Nano%20Letters.pdf?sequence=1)). Using the same data, this notebook reproduces the main results of this article.

###### Requirements

Hyperspy 0.8.5

###### Notebook

Notebook originally developed for the Hyperspy workshop at the University of Cambridge on 13 April 2015. It was updated on 5 May 2016 for Python 3 compatibility and various developments in HyperSpy.

###### Data

The sample and the data used in this tutorial are described in 
D. Roussow, et al., Nano Letters, In Press (2015) (see the [full article](https://www.repository.cam.ac.uk/bitstream/handle/1810/248102/Roussouw%20et%20al%202015%20Nano%20Letters.pdf?sequence=1)).

In [ ]:
#Download the data (1MB)
from urllib.request import urlretrieve, urlopen
fnames= ['bare_core.hdf5', 'core_shell.hdf5']   
url = 'http://cook.msm.cam.ac.uk//~hyperspy//EDS_tutorial//'
files = [urlretrieve(url + fname, fname) for fname in fnames]

###### Sample

FePt@Fe$_3$O$_4$ core-shell nanoparticles are investigated with an EDS/TEM experiment (FEI Osiris TEM, 4 EDS detectors). The composition of the core can be measured with ICA (see figure 1c). To prove the accuracy of the results, measurments on bare FePt bimetallic nanoparticles from a synthesis prior to the shell addition step are used.

<img src="images/core_shell.png" style="height:350px;">
Figure 1: (a) A spectrum image obtained from a cluster of core-shell nanoparticles. (b) The nanoparticles are comprised of a bi-metallic Pt/Fe core surrounded by an iron oxide shell on a carbon support. (c) ICA decomposes the mixed EDX signals into components representing the core (IC#0), shell (IC#1) and support (IC#2).

###### Author

Pierre Burdet (updated by Duncan Johnstone)

###### <a id='top'></a> Table of contents

1. <a href='#loa'> Loading</a>
2. <a href='#bss'> Blind source separation of core/shell nanoparticles</a>
3. <a href='#bare'> Representative spectrum from bare cores</a>
4. <a href='#com'> Comparison and quantification</a>
5. <a href='#fur'> Going futher: Isolating the nanoparticles</a>

Import HyperSpy, numpy and matplotlib libraries

In [ ]:
%matplotlib qt4
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt

###### For a nice on-line display

In [ ]:
%matplotlib inline

## <a id='loa'></a> 1. Loading

<a href='#top'> Table of contents</a>

Load the spectrum images of the bare seeds and the core shell nanoparticles.

In [ ]:
c = hs.load('bare_core.hdf5')
cs = hs.load('core_shell.hdf5')

In [ ]:
c.metadata

Plot the intensity of Fe K${\alpha}$ and Pt L${\alpha}$.

In [ ]:
axes = hs.plot.plot_images(c.get_lines_intensity() + cs.get_lines_intensity(),
                           scalebar='all', axes_decor=None, per_row=2, cmap='RdBu')

## <a id='bss'></a> 2. Blind source separation of core/shell nanoparticles

<a href='#top'> Table of contents</a>

Apply blind source separation (ICA) to obtain a factor (spectrum) corresponding to the core.

In [ ]:
cs.change_dtype('float')
cs.decomposition()

In [ ]:
ax = cs.plot_explained_variance_ratio()

ICA on the three first components.

In [ ]:
cs.blind_source_separation(3)

In [ ]:
axes = hs.plot.plot_images(cs.get_bss_loadings(), axes_decor=None, cmap='RdBu')

In [ ]:
axes = hs.plot.plot_spectra(cs.get_bss_factors())

The first component corresponds to the core.

In [ ]:
s_bss = cs.get_bss_factors()[0]

## <a id='bare'></a> 3. Representative spectrum from bare cores

<a href='#top'> Table of contents</a>

To obtain a representative spectrum of the bare nanoparticles, the low intensity of Pt L${\alpha}$ is masked.

In [ ]:
pt_la = c.get_lines_intensity(['Pt_La'])[0]
mask = pt_la > 6

In [ ]:
axes = hs.plot.plot_images([mask, pt_la * mask], axes_decor=None, colorbar=None,
                           label=['Mask', 'Pt L${\\alpha}$ intensity'], cmap='RdBu')

To apply the mask, the navigation dimensions of the mask must be manipulated to match the navigation dimensions of the EDS spectrum image. This is achived crudely via first generating a mask using the built in vacuum_mask() method and then overwriting the data with the mask generated above.

In [ ]:
c_mask = c.vacuum_mask()
c_mask.data = mask.data

The sum over the particles is used as a bare core spectrum.

In [ ]:
s_bare = (c * c_mask).sum(0).sum(0)

## <a id='com'></a> 4. Comparison and quantification

<a href='#top'> Table of contents</a>

Stack together the spectrum of bare particles and the first ICA component.

In [ ]:
s_bare.change_dtype('float')
s = hs.stack([s_bare, s_bss], new_axis_name='Bare or BSS')
s.metadata.General.title = 'Bare or BSS'

In [ ]:
axes = hs.plot.plot_spectra(s, style='mosaic', legend=['Bare particles', 'BSS #0'])

X-ray intensities measurement with background subtraction.

In [ ]:
w = s.estimate_background_windows()

In [ ]:
s.plot(background_windows=w)

Refinement of the windows position.

In [ ]:
w

In [ ]:
w[1, 0] = 8.44
w[1, 1] = 8.65

In [ ]:
s.plot(background_windows=w, navigator='slider')

In [ ]:
sI = s.get_lines_intensity(background_windows=w)

Set up the kfactors for Fe K${\alpha}$ and Pt L${\alpha}$.

In [ ]:
#From Brucker software (Esprit)
kfactors = [1.450226, 5.075602]

Quantify with Cliff Lorimer.

In [ ]:
composition = s.quantification(intensities=sI, kfactors=kfactors,
                 plot_result=True, composition_units='atomic')

## <a id='fur'></a> 5. Going further

<a href='#top'> Table of contents</a>

### 5.1. Isolating the nanoparticles

Further image processing with [scikit-image](http://scikit-image.org/) and [scipy](http://www.scipy.org/). Apply a watershed transformation to isolate the nanoparticles.

- Transform the mask into a distance map.
- Find local maxima.
- Apply the watershed to the distance map using the local maximum as seed (markers).

Adapted from this scikit-image [example](http://scikit-image.org/docs/dev/auto_examples/plot_watershed.html).

In [ ]:
from scipy.ndimage import distance_transform_edt, label
from skimage.morphology import watershed
from skimage.feature import peak_local_max

In [ ]:
distance = distance_transform_edt(mask.data)
local_maxi = peak_local_max(distance, indices=False,
                            min_distance=2, labels=mask.data)
labels = watershed(-distance, markers=label(local_maxi)[0],
                   mask=mask.data)

In [ ]:
axes = hs.plot.plot_images(
    [pt_la, mask, hs.signals.Image(distance), hs.signals.Image(labels)],
    axes_decor='off', per_row=2, colorbar=None, cmap='RdYlBu_r',
    label=['Pt L${\\alpha}$ intensity', 'Mask',
           'Distances', 'Separated particles'])